* Issue of multiple feature on one location 
  * Had to apply set to avoid this issue

In [3]:
import pandas as pd
from collections import Counter
from utils import parse_cdhit, replace_val
import itertools
import numpy as np


In [6]:
data = pd.read_table("data/ENA_ML_input_ORFs")
data.head(2)

,Feature_id,contig_id,position_start,position_end,ORF_location_on_contig,annotation_val
0,ENA_AY095314_AY095314.2_3724_3921_10,AY095314.2,3724,3921,10,Unknown
1,ENA_JX889246_JX889246.1_45181_44900_68,JX889246.1,45181,44900,68,Uncharacterized protein


In [7]:
seq_to_clust = parse_cdhit("data/ENA.40.clstr")
list(seq_to_clust.items())[0:2]

[('ENA_KP211958_KP211958.1_65082_87137_88', '0'),
 ('ENA_AY940168_AY940168.2_57144_79082_90', '1')]

In [8]:
## Add a column to show the cluster to which each feature_id belongs
data["feature_number"] = replace_val(data.Feature_id, seq_to_clust)
data.feature_number = data.feature_number.astype(int)
data.head(2)

,Feature_id,contig_id,position_start,position_end,ORF_location_on_contig,annotation_val,feature_number
0,ENA_AY095314_AY095314.2_3724_3921_10,AY095314.2,3724,3921,10,Unknown,57065
1,ENA_JX889246_JX889246.1_45181_44900_68,JX889246.1,45181,44900,68,Uncharacterized protein,43194


In [9]:
# Add a contig sequential number in addition to contig id.
contig_id_to_int = {y:x for x,y in enumerate(data.contig_id.tolist())}
#### data.replace({'contig_id': contig_id_to_int})
data["contig_number"] = replace_val(data.contig_id, contig_id_to_int)
data.head(2)

,Feature_id,contig_id,position_start,position_end,ORF_location_on_contig,annotation_val,feature_number,contig_number
0,ENA_AY095314_AY095314.2_3724_3921_10,AY095314.2,3724,3921,10,Unknown,57065,237296
1,ENA_JX889246_JX889246.1_45181_44900_68,JX889246.1,45181,44900,68,Uncharacterized protein,43194,234989


#### Testing code with AB012574.1

In [20]:
x = data[data.contig_id == "AB012574.1"].sort_values(by="ORF_location_on_contig")
x

,Feature_id,contig_id,position_start,position_end,ORF_location_on_contig,annotation_val,feature_number,contig_number
163381,ENA_AB012574_AB012574.1_1_591_1,AB012574.1,1,591,1,Phage replication protein,18804,221381
165466,ENA_AB012574_AB012574.1_595_948_2,AB012574.1,595,948,2,Unknown,34298,221381
169624,ENA_AB012574_AB012574.1_960_1205_3,AB012574.1,960,1205,3,Unknown,48370,221381
129210,ENA_AB012574_AB012574.1_1216_1449_4,AB012574.1,1216,1449,4,Unknown,50392,221381
31977,ENA_AB012574_AB012574.1_1583_3058_5,AB012574.1,1583,3058,5,Unknown,4917,221381
110108,ENA_AB012574_AB012574.1_3060_3374_6,AB012574.1,3060,3374,6,Unknown,39228,221381
221381,ENA_AB012574_AB012574.1_3371_4513_7,AB012574.1,3371,4513,7,Unknown,7275,221381
60438,ENA_AB012574_AB012574.1_5516_4821_8,AB012574.1,5516,4821,8,Unknown,15178,221381
28717,ENA_AB012574_AB012574.1_6049_5696_9,AB012574.1,6049,5696,9,Unknown,32777,221381
177073,ENA_AB012574_AB012574.1_6488_6042_10,AB012574.1,6488,6042,10,Unknown,25521,221381


In [73]:
def sort_pair(pair):
    if pair[0] > pair[1]:
        return (pair[1], pair[0])
    else:
        return pair
# window_size is max. If contig contains less, window_size will be 
# adjusted accordingly.
def find_pairs_in_window(group, window_size=8):
    unsorted_pairs = []
    pairs=[]
    contig_features = group.feature_number.values
    window_size = min(len(contig_features), window_size)
    for i, line_id in enumerate(contig_features[:-(window_size-1)]):
        unsorted_pairs.append(list(itertools.combinations(contig_features[i: i+window_size], 2)))
    for pair in set(list(itertools.chain(*unsorted_pairs))):
        pairs.append(sort_pair(pair))
    return list(set(pairs))

In [74]:
data.groupby("contig_number").apply(find_pairs_in_window).apply(lambda x: len(x))

contig_number
82638       15
106079       6
109093      15
118155       3
122079      15
          ... 
237325     546
237326    1463
237327     406
237328     350
237329     378
Length: 2480, dtype: int64